# Main

In [1]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dense, Activation, GlobalAvgPool2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 기본적으로 CIFAR-10을 사용해 구현
from keras.datasets import cifar10
# DenseNet과 스코어 점수를 비교하기위한 MNIST 데이터셋
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

In [2]:
class PlainBlock(keras.layers.Layer):
  def __init__(self, filters, strides = 1, activation = 'relu', **kwargs):
    # 이 블록을 객체화 할때 내가 지정해준 매개변수 이외의 것을 지정해주면
    # 부모 클래스 keras.layers.Layer 이곳으로 그 매개변수를 전달함
    super().__init__(**kwargs)

    # get_config를 위해 변수 초기화
    self.filters = filters
    self.strides = strides
    self.activation = keras.activations.get(activation)

    self.main_layers = [
        # 커널의 사이즈는 3x3으로 고정 
        # strides는 풀링을 대체해 2로 받아야 할 때도 있기 때문에 변수로 받고
        # 입출력의 사이즈를 동일하게 해주기 위해 패딩은 same으로
        # 컨볼루션을 진행 할때마다 배치 정규화와 Relu 함수를 통과해 진행
        # bias는 BN을 진행하면서 bias를 더해주기 때문에 Conv2d안에서는 False로 지정
        keras.layers.Conv2D(filters, kernel_size=3, strides = strides, padding='same', use_bias=False),
        keras.layers.BatchNormalization(),
        self.activation,
        keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias=False),
        keras.layers.BatchNormalization(),
    ]

  # ModelCheckpoint를 사용하기위해 각 변수를 딕셔너리로 반환
  def get_config(self):
    config = super().get_config()
    config.update({
        "filters": self.filters,
        "strides": self.strides,
        "activation": self.activation
    })
    return config
  
  # 실제로 레이어가 어떻게 구동이 되는지 정해주는 call 선언
  # add로 객체화가 되면 이 call함수가 호출
  def call(self, input):
    x = input
    for layer in self.main_layers:
      x = layer(x)
    return self.activation(x)

In [3]:
class ResBlock(keras.layers.Layer):
  def __init__(self, filters, strides = 1, activation = 'relu', **kwargs):
    # 이 블록을 객체화 할때 내가 지정해준 매개변수 이외의 것을 지정해주면
    # 부모 클래스 keras.layers.Layer 이곳으로 그 매개변수를 전달함
    super().__init__(**kwargs)

    # get_config를 위해 변수 초기화
    self.filters = filters
    self.strides = strides
    self.activation = keras.activations.get(activation)

    self.main_layers = [
        # 커널의 사이즈는 3x3으로 고정 
        # strides는 풀링을 대체해 2로 받아야 할 때도 있기 때문에 변수로 받고
        # 입출력의 사이즈를 동일하게 해주기 위해 패딩은 same으로
        # 컨볼루션을 진행 할때마다 배치 정규화와 Relu 함수를 통과해 진행
        # bias는 BN을 진행하면서 bias를 더해주기 때문에 Conv2d안에서는 False로 지정
        keras.layers.Conv2D(filters, kernel_size=3, strides = strides, padding='same', use_bias=False),
        keras.layers.BatchNormalization(),
        self.activation,
        keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias=False),
        keras.layers.BatchNormalization(),
    ]
    self.skip_layers = []
    if strides != 1:
      self.skip_layers = [
          keras.layers.Conv2D(filters, 1, strides=strides, padding='same', use_bias=False),
          keras.layers.BatchNormalization()
      ]
  # ModelCheckpoint를 사용하기위해 각 변수를 딕셔너리로 반환
  def get_config(self):
    config = super().get_config()
    config.update({
        "filters": self.filters,
        "strides": self.strides,
        "activation": self.activation
    })
    return config

  # 실제로 레이어가 어떻게 구동이 되는지 정해주는 call 선언
  # 이곳에서 인풋을 더해주는 작업을 통해 PlainBlock과의 점수 비교
  # add로 객체화가 되면 이 call함수가 호출
  def call(self, input):
    x = input
    for layer in self.main_layers:
      x = layer(x)
    skip_x = input
    for layer in self.skip_layers:
      skip_x = layer(skip_x)
    return self.activation(x + skip_x)

In [ ]:
# class ResNet(keras.layers.Layer):
#   def __init__(self, block, num_blocks, **kwargs):
#     super().__init__(**kwargs)
#     self.prev_filters = 16
#     self.activation = keras.activations.get('softmax')

#     self.all_layers=[
#         keras.layers.Conv2D(16, 3, input_shape=(32, 32, 3), padding='same', use_bias=False),
#         keras.layers.BatchNormalization(),
#         self._make_layer(block, 16, num_blocks[0], stride=1),
#         self._make_layer(block, 32, num_blocks[1], stride=1),
#         self._make_layer(block, 64, num_blocks[2], stride=1),
#         keras.layers.GlobalAvgPool2D(),
#         keras.layers.Dense(10),
#     ]

#   def _make_layer(self, block, filters, num_blocks, stride):
#     strides = [stride] + [1] * (num_blocks - 1)
#     layers = []
#     for stride in strides:
#       layers.append(block(filters, stride))
#       self.prev_filters = filters # 다음 레이어를 위해 채널 수 변경
#     return layers
  
#   def call(self, input):
#     x = input
#     for layer in self.all_layers:
#       x = layer(x)

#     return self.activation(x)
  

# CIFAR-10 Dataset

In [4]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 4s 0us/step


In [5]:
# 데이터의 기본 타깃값은 0~9로 설정되어있음
# 논문에선 레이어의 마지막에 분류하고 싶은 클래스의 수만큼
# 카테고리 분류를 해주기 때문에 원핫 인코딩이 필요
y_test

array([[3],
       [8],
       [8],
       ...,
       [5],
       [1],
       [7]], dtype=uint8)

In [6]:
# 원핫 인코딩 진행
import sklearn.preprocessing

label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range((max(y_train)+1)[0]))
y_train = label_binarizer.transform(y_train)
label_binarizer.fit(range((max(y_test)+1)[0]))
y_test = label_binarizer.transform(y_test)
print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


In [7]:
# 원핫 인코딩 완료
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

# PlainBlock-14layers

In [8]:
model1 = Sequential()
model1.add(Conv2D(16, 3, input_shape=(32, 32, 3), padding='same', use_bias=False))
model1.add(BatchNormalization())
prev_filters = 16
# 채널수를 16, 32, 64 로 증가
for filters in [16]*2+[32]*2+[64]*2:
  if filters == prev_filters:
    strides = 1
  else:
    strides = 2
  model1.add(PlainBlock(filters, strides=strides, activation = 'relu'))
  prev_filters = filters

model1.add(GlobalAvgPool2D())
model1.add(Dense(10, activation='softmax'))
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 16)        432       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 16)       64        
 ormalization)                                                   
                                                                 
 plain_block (PlainBlock)    (None, 32, 32, 16)        4736      
                                                                 
 plain_block_1 (PlainBlock)  (None, 32, 32, 16)        4736      
                                                                 
 plain_block_2 (PlainBlock)  (None, 16, 16, 32)        14080     
                                                                 
 plain_block_3 (PlainBlock)  (None, 16, 16, 32)        18688     
                                                        

In [9]:
# optimizer를 SGD에 learning_rate를 0.01, momentum을 0.9로 지정
model1.compile(optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])

# EarlyStopping 의 patience를 20으로 설정해주엇기 때문에 사실상 기능을 꺼둔 상태
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=20)
plane14_path = 'Plane14.hdf5'
checkpointer = ModelCheckpoint(filepath = plane14_path, monitor='val_loss', verbose=0, save_best_only=True)

In [10]:
histroy = model1.fit(X_train, y_train, validation_split = 0.25, epochs = 20, batch_size = 128, callbacks=[early_stopping_callback, checkpointer])

Epoch 1/20
293/293 [==============================] - 18s 23ms/step - loss: 1.6487 - accuracy: 0.3883 - val_loss: 1.5672 - val_accuracy: 0.4222
Epoch 2/20
293/293 [==============================] - 6s 19ms/step - loss: 1.2016 - accuracy: 0.5657 - val_loss: 2.5011 - val_accuracy: 0.2668
Epoch 3/20
293/293 [==============================] - 6s 19ms/step - loss: 0.9961 - accuracy: 0.6436 - val_loss: 1.2838 - val_accuracy: 0.5526
Epoch 4/20
293/293 [==============================] - 6s 19ms/step - loss: 0.8694 - accuracy: 0.6925 - val_loss: 1.1226 - val_accuracy: 0.5991
Epoch 5/20
293/293 [==============================] - 6s 20ms/step - loss: 0.7813 - accuracy: 0.7234 - val_loss: 1.6012 - val_accuracy: 0.5594
Epoch 6/20
293/293 [==============================] - 6s 20ms/step - loss: 0.6933 - accuracy: 0.7581 - val_loss: 0.9580 - val_accuracy: 0.6701
Epoch 7/20
293/293 [==============================] - 6s 20ms/step - loss: 0.6371 - accuracy: 0.7770 - val_loss: 0.7937 - val_accuracy: 0.728

In [11]:
model1.load_weights(plane14_path)
model1.evaluate(X_test, y_test)[1]

313/313 [==============================] - 1s 4ms/step - loss: 0.8092 - accuracy: 0.7192


0.7192000150680542

# ResBlock-14layers

In [ ]:
model2 = Sequential()
model2.add(Conv2D(16, 3, input_shape=(32, 32, 3), padding='same', use_bias=False))
model2.add(BatchNormalization())
prev_filters = 16
# 채널수를 16, 32, 64 로 증가
for filters in [16]*2+[32]*2+[64]*2:
  if filters == prev_filters:
    strides = 1
  else:
    strides = 2
  model2.add(ResBlock(filters, strides=strides))
  prev_filters = filters
model2.add(GlobalAvgPool2D())
model2.add(Dense(10, activation='softmax'))
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_63 (Conv2D)          (None, 32, 32, 16)        432       
                                                                 
 batch_normalization_63 (Bat  (None, 32, 32, 16)       64        
 chNormalization)                                                
                                                                 
 res_block (ResBlock)        (None, 32, 32, 16)        4736      
                                                                 
 res_block_1 (ResBlock)      (None, 32, 32, 16)        4736      
                                                                 
 res_block_2 (ResBlock)      (None, 16, 16, 32)        14720     
                                                                 
 res_block_3 (ResBlock)      (None, 16, 16, 32)        18688     
                                                      

In [ ]:
model2.compile(optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=20)
res14_path = 'Res14.hdf5'
checkpointer = ModelCheckpoint(filepath = res14_path, monitor='val_loss', verbose=0, save_best_only=True)

In [ ]:
model2.fit(X_train, y_train, validation_split = 0.25, epochs = 20, batch_size = 128, callbacks=[early_stopping_callback, checkpointer])

Epoch 1/20
293/293 [==============================] - 8s 23ms/step - loss: 1.5513 - accuracy: 0.4296 - val_loss: 2.3173 - val_accuracy: 0.3036
Epoch 2/20
293/293 [==============================] - 6s 22ms/step - loss: 1.1462 - accuracy: 0.5829 - val_loss: 1.3266 - val_accuracy: 0.5332
Epoch 3/20
293/293 [==============================] - 6s 22ms/step - loss: 0.9743 - accuracy: 0.6493 - val_loss: 1.2996 - val_accuracy: 0.5658
Epoch 4/20
293/293 [==============================] - 6s 22ms/step - loss: 0.8589 - accuracy: 0.6950 - val_loss: 1.2422 - val_accuracy: 0.5751
Epoch 5/20
293/293 [==============================] - 6s 22ms/step - loss: 0.7708 - accuracy: 0.7263 - val_loss: 1.4514 - val_accuracy: 0.5562
Epoch 6/20
293/293 [==============================] - 6s 22ms/step - loss: 0.6980 - accuracy: 0.7523 - val_loss: 1.0953 - val_accuracy: 0.6280
Epoch 7/20
293/293 [==============================] - 6s 22ms/step - loss: 0.6368 - accuracy: 0.7744 - val_loss: 1.1243 - val_accuracy: 0.6376

In [ ]:
model2.load_weights(res14_path)
model2.evaluate(X_test, y_test)[1]

313/313 [==============================] - 2s 5ms/step - loss: 0.9728 - accuracy: 0.7095


0.7095000147819519

# PlainBlock-18layers

In [ ]:
model3 = Sequential()
model3.add(Conv2D(64, 3, input_shape=(32, 32, 3), padding='same', use_bias=False))
model3.add(BatchNormalization())
prev_filters = 64
# 채널수를 64, 128, 256, 512 로 증가
for filters in [64]*2+[128]*2+[256]*2+[512]*2:
  if filters == prev_filters:
    strides = 1
  else:
    strides = 2
  model3.add(PlainBlock(filters, strides=strides))
  prev_filters = filters

model3.add(GlobalAvgPool2D())
model3.add(Dense(10, activation='softmax'))
model3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 32, 32, 64)        1728      
                                                                 
 batch_normalization_13 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                                 
 plane_block_6 (PlaneBlock)  (None, 32, 32, 64)        74240     
                                                                 
 plane_block_7 (PlaneBlock)  (None, 32, 32, 64)        74240     
                                                                 
 plane_block_8 (PlaneBlock)  (None, 16, 16, 128)       222208    
                                                                 
 plane_block_9 (PlaneBlock)  (None, 16, 16, 128)       295936    
                                                      

In [ ]:
model3.compile(optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=20)
Plain18_path = 'Plain18.hdf5'
checkpointer = ModelCheckpoint(filepath = Plain18_path, monitor='val_loss', verbose=0, save_best_only=True)

In [ ]:
model3.fit(X_train, y_train, validation_split = 0.25, epochs = 20, batch_size = 128, callbacks=[early_stopping_callback, checkpointer])

Epoch 1/20
293/293 [==============================] - 33s 113ms/step - loss: 1.4304 - accuracy: 0.4801 - val_loss: 1.8097 - val_accuracy: 0.4092
Epoch 2/20
293/293 [==============================] - 32s 108ms/step - loss: 0.9543 - accuracy: 0.6595 - val_loss: 1.2093 - val_accuracy: 0.5593
Epoch 3/20
293/293 [==============================] - 31s 106ms/step - loss: 0.6897 - accuracy: 0.7582 - val_loss: 0.9747 - val_accuracy: 0.6746
Epoch 4/20
293/293 [==============================] - 31s 106ms/step - loss: 0.4991 - accuracy: 0.8260 - val_loss: 0.8851 - val_accuracy: 0.7144
Epoch 5/20
293/293 [==============================] - 31s 106ms/step - loss: 0.3448 - accuracy: 0.8797 - val_loss: 1.3003 - val_accuracy: 0.6347
Epoch 6/20
293/293 [==============================] - 31s 105ms/step - loss: 0.2431 - accuracy: 0.9163 - val_loss: 1.1744 - val_accuracy: 0.6910
Epoch 7/20
293/293 [==============================] - 31s 106ms/step - loss: 0.1572 - accuracy: 0.9467 - val_loss: 0.9808 - val_ac

In [ ]:
model3.load_weights(Plain18_path)
model3.evaluate(X_test, y_test)[1]

313/313 [==============================] - 4s 11ms/step - loss: 0.9155 - accuracy: 0.7069


0.7069000005722046

# ResBlock-18layers

In [ ]:
model4 = Sequential()
model4.add(Conv2D(64, 3, input_shape=(32, 32, 3), padding='same', use_bias=False))
model4.add(BatchNormalization())
prev_filters = 64
for filters in [64]*2+[128]*2+[256]*2+[512]*2:
  if filters == prev_filters:
    strides = 1
  else:
    strides = 2
  model4.add(ResBlock(filters, strides=strides))
  prev_filters = filters

model4.add(GlobalAvgPool2D())
model4.add(Dense(10, activation='softmax'))
model4.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_206 (Conv2D)         (None, 32, 32, 64)        1728      
                                                                 
 batch_normalization_206 (Ba  (None, 32, 32, 64)       256       
 tchNormalization)                                               
                                                                 
 res_block_62 (ResBlock)     (None, 32, 32, 64)        74240     
                                                                 
 res_block_63 (ResBlock)     (None, 32, 32, 64)        74240     
                                                                 
 res_block_64 (ResBlock)     (None, 16, 16, 128)       230912    
                                                                 
 res_block_65 (ResBlock)     (None, 16, 16, 128)       295936    
                                                      

In [ ]:
model4.compile(optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=20)
res18_path = 'Res18.hdf5'
checkpointer = ModelCheckpoint(filepath = res18_path, monitor='val_loss', verbose=0, save_best_only=True)

In [ ]:
model4.fit(X_train, y_train, validation_split = 0.25, epochs = 20, batch_size = 128, callbacks=[early_stopping_callback, checkpointer])

Epoch 1/20
293/293 [==============================] - 36s 117ms/step - loss: 1.4119 - accuracy: 0.4899 - val_loss: 1.8148 - val_accuracy: 0.3886
Epoch 2/20
293/293 [==============================] - 34s 116ms/step - loss: 0.9048 - accuracy: 0.6763 - val_loss: 1.2323 - val_accuracy: 0.5591
Epoch 3/20
293/293 [==============================] - 33s 114ms/step - loss: 0.6180 - accuracy: 0.7822 - val_loss: 1.2474 - val_accuracy: 0.6247
Epoch 4/20
293/293 [==============================] - 34s 116ms/step - loss: 0.4092 - accuracy: 0.8553 - val_loss: 1.1190 - val_accuracy: 0.6534
Epoch 5/20
293/293 [==============================] - 33s 114ms/step - loss: 0.2398 - accuracy: 0.9185 - val_loss: 1.7441 - val_accuracy: 0.6097
Epoch 6/20
293/293 [==============================] - 33s 114ms/step - loss: 0.1386 - accuracy: 0.9526 - val_loss: 1.6600 - val_accuracy: 0.6360
Epoch 7/20
293/293 [==============================] - 33s 114ms/step - loss: 0.0935 - accuracy: 0.9685 - val_loss: 1.5198 - val_ac

In [ ]:
model4.load_weights(res18_path)
model4.evaluate(X_test, y_test)[1]

313/313 [==============================] - 4s 12ms/step - loss: 1.0288 - accuracy: 0.7892


0.7892000079154968

# PlainBlock-34layers

In [ ]:
model5 = Sequential()
model5.add(Conv2D(64, 3, input_shape=(32, 32, 3), padding='same', use_bias=False))
model5.add(BatchNormalization())
prev_filters = 64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
  if filters == prev_filters:
    strides = 1
  else:
    strides = 2
  model5.add(PlainBlock(filters, strides=strides))
  prev_filters = filters

model5.add(GlobalAvgPool2D())
model5.add(Dense(10, activation='softmax'))
model5.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 32, 32, 64)        1728      
                                                                 
 batch_normalization_30 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                                 
 plane_block_14 (PlaneBlock)  (None, 32, 32, 64)       74240     
                                                                 
 plane_block_15 (PlaneBlock)  (None, 32, 32, 64)       74240     
                                                                 
 plane_block_16 (PlaneBlock)  (None, 32, 32, 64)       74240     
                                                                 
 plane_block_17 (PlaneBlock)  (None, 16, 16, 128)      222208    
                                                      

In [ ]:
model5.compile(optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=20)
Plain34_path = 'Plain34.hdf5'
checkpointer = ModelCheckpoint(filepath = Plain34_path, monitor='val_loss', verbose=0, save_best_only=True)

In [ ]:
model5.fit(X_train, y_train, validation_split = 0.25, epochs = 20, batch_size = 128, callbacks=[early_stopping_callback, checkpointer])

Epoch 1/20
293/293 [==============================] - 58s 190ms/step - loss: 1.9853 - accuracy: 0.2627 - val_loss: 1.8202 - val_accuracy: 0.3001
Epoch 2/20
293/293 [==============================] - 55s 186ms/step - loss: 1.6550 - accuracy: 0.3851 - val_loss: 1.6756 - val_accuracy: 0.3950
Epoch 3/20
293/293 [==============================] - 55s 188ms/step - loss: 1.4832 - accuracy: 0.4563 - val_loss: 2.2192 - val_accuracy: 0.3110
Epoch 4/20
293/293 [==============================] - 55s 189ms/step - loss: 1.3289 - accuracy: 0.5154 - val_loss: 1.4490 - val_accuracy: 0.4935
Epoch 5/20
293/293 [==============================] - 55s 188ms/step - loss: 1.1753 - accuracy: 0.5770 - val_loss: 1.3003 - val_accuracy: 0.5352
Epoch 6/20
293/293 [==============================] - 55s 189ms/step - loss: 1.0368 - accuracy: 0.6317 - val_loss: 1.0526 - val_accuracy: 0.6290
Epoch 7/20
293/293 [==============================] - 55s 188ms/step - loss: 0.9102 - accuracy: 0.6798 - val_loss: 1.0511 - val_ac

In [ ]:
model5.load_weights(Plain34_path)
model5.evaluate(X_test, y_test)[1]

313/313 [==============================] - 6s 19ms/step - loss: 0.8959 - accuracy: 0.7215


0.7214999794960022

# ResBlock-34layers

In [ ]:
model6 = Sequential()
model6.add(Conv2D(64, 3, input_shape=(32, 32, 3), padding='same', use_bias=False))
model6.add(BatchNormalization())
prev_filters = 64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
  if filters == prev_filters:
    strides = 1
  else:
    strides = 2
  model6.add(ResBlock(filters, strides=strides))
  prev_filters = filters

model6.add(GlobalAvgPool2D())
model6.add(Dense(10, activation='softmax'))
model6.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_226 (Conv2D)         (None, 32, 32, 64)        1728      
                                                                 
 batch_normalization_226 (Ba  (None, 32, 32, 64)       256       
 tchNormalization)                                               
                                                                 
 res_block_70 (ResBlock)     (None, 32, 32, 64)        74240     
                                                                 
 res_block_71 (ResBlock)     (None, 32, 32, 64)        74240     
                                                                 
 res_block_72 (ResBlock)     (None, 32, 32, 64)        74240     
                                                                 
 res_block_73 (ResBlock)     (None, 16, 16, 128)       230912    
                                                      

In [ ]:
model6.compile(optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=20, restore_best_weights = True)
res34_path = 'Res34.hdf5'
checkpointer = ModelCheckpoint(filepath = res34_path, monitor='val_loss', verbose=0, save_best_only=True)

In [ ]:
model6.fit(X_train, y_train, validation_split = 0.25, epochs = 20, batch_size = 128, callbacks=[early_stopping_callback, checkpointer])

Epoch 1/20
293/293 [==============================] - 58s 198ms/step - loss: 1.0102 - accuracy: 0.6422 - val_loss: 0.9719 - val_accuracy: 0.6613
Epoch 2/20
293/293 [==============================] - 57s 196ms/step - loss: 0.7077 - accuracy: 0.7510 - val_loss: 1.1852 - val_accuracy: 0.6258
Epoch 3/20
293/293 [==============================] - 57s 196ms/step - loss: 0.5043 - accuracy: 0.8255 - val_loss: 1.3695 - val_accuracy: 0.6158
Epoch 4/20
293/293 [==============================] - 58s 196ms/step - loss: 0.3390 - accuracy: 0.8799 - val_loss: 1.1818 - val_accuracy: 0.6770
Epoch 5/20
293/293 [==============================] - 58s 196ms/step - loss: 0.2222 - accuracy: 0.9214 - val_loss: 1.5325 - val_accuracy: 0.6554
Epoch 6/20
293/293 [==============================] - 58s 197ms/step - loss: 0.1571 - accuracy: 0.9449 - val_loss: 1.6992 - val_accuracy: 0.6509
Epoch 7/20
293/293 [==============================] - 58s 196ms/step - loss: 0.1260 - accuracy: 0.9553 - val_loss: 2.2300 - val_ac

In [ ]:
model6.load_weights(res34_path)
model6.evaluate(X_test, y_test)[1]

313/313 [==============================] - 6s 20ms/step - loss: 1.2674 - accuracy: 0.7920


0.7919999957084656

# 비교

Plain 14 \\
0.7192000150680542 \\
Res 14 \\
0.7095000147819519 \\

Plain 18 \\
0.7069000005722046 \\
Res 18 \\
0.7892000079154968

Plain 34 \\
0.7214999794960022 \\
Res	34 \\
0.7919999957084656

# MNIST Dataset ResBlock-32layers

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
y_train.shape

(60000,)

In [ ]:
import sklearn.preprocessing

label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range((max(y_train)+1)))
y_train = label_binarizer.transform(y_train)
label_binarizer.fit(range((max(y_test)+1)))
y_test = label_binarizer.transform(y_test)
print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


In [ ]:
model7 = Sequential()
model7.add(Conv2D(64, 3, input_shape=(28, 28, 1), padding='same', use_bias=False))
model7.add(BatchNormalization())
prev_filters = 64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
  if filters == prev_filters:
    strides = 1
  else:
    strides = 2
  model7.add(ResBlock(filters, strides=strides))
  prev_filters = filters

model7.add(GlobalAvgPool2D())
model7.add(Dense(10, activation='softmax'))
model7.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_154 (Conv2D)         (None, 28, 28, 64)        576       
                                                                 
 batch_normalization_154 (Ba  (None, 28, 28, 64)       256       
 tchNormalization)                                               
                                                                 
 res_block_68 (ResBlock)     (None, 28, 28, 64)        74240     
                                                                 
 res_block_69 (ResBlock)     (None, 28, 28, 64)        74240     
                                                                 
 res_block_70 (ResBlock)     (None, 28, 28, 64)        74240     
                                                                 
 res_block_71 (ResBlock)     (None, 14, 14, 128)       230912    
                                                      

In [ ]:
model7.compile(optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10, restore_best_weights = True)

In [ ]:
model7.fit(X_train, y_train, validation_split = 0.25, epochs = 10, batch_size = 128, callbacks=[early_stopping_callback])

Epoch 1/10
352/352 [==============================] - 118s 187ms/step - loss: 0.1633 - accuracy: 0.9484 - val_loss: 0.0857 - val_accuracy: 0.9725
Epoch 2/10
352/352 [==============================] - 64s 183ms/step - loss: 0.0286 - accuracy: 0.9913 - val_loss: 0.1133 - val_accuracy: 0.9673
Epoch 3/10
352/352 [==============================] - 64s 182ms/step - loss: 0.0146 - accuracy: 0.9953 - val_loss: 0.0555 - val_accuracy: 0.9836
Epoch 4/10
352/352 [==============================] - 64s 183ms/step - loss: 0.0073 - accuracy: 0.9978 - val_loss: 0.0680 - val_accuracy: 0.9821
Epoch 5/10
352/352 [==============================] - 65s 183ms/step - loss: 0.0025 - accuracy: 0.9993 - val_loss: 0.0350 - val_accuracy: 0.9914
Epoch 6/10
352/352 [==============================] - 64s 182ms/step - loss: 8.1540e-04 - accuracy: 0.9999 - val_loss: 0.0340 - val_accuracy: 0.9921
Epoch 7/10
352/352 [==============================] - 64s 183ms/step - loss: 2.8325e-04 - accuracy: 1.0000 - val_loss: 0.0323

In [ ]:
model7.evaluate(X_test, y_test)[1]

313/313 [==============================] - 6s 19ms/step - loss: 0.0195 - accuracy: 0.9941


0.9940999746322632

In [ ]:
model7.compile(optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model7.fit(X_train, y_train, epochs = 10, batch_size = 128)

Epoch 1/10
469/469 [==============================] - 81s 168ms/step - loss: 0.1289 - accuracy: 0.9589
Epoch 2/10
469/469 [==============================] - 78s 167ms/step - loss: 0.0238 - accuracy: 0.9928
Epoch 3/10
469/469 [==============================] - 78s 166ms/step - loss: 0.0124 - accuracy: 0.9961
Epoch 4/10
469/469 [==============================] - 78s 167ms/step - loss: 0.0066 - accuracy: 0.9982
Epoch 5/10
469/469 [==============================] - 78s 167ms/step - loss: 0.0026 - accuracy: 0.9993
Epoch 6/10
469/469 [==============================] - 78s 167ms/step - loss: 0.0028 - accuracy: 0.9993
Epoch 7/10
469/469 [==============================] - 78s 167ms/step - loss: 0.0011 - accuracy: 0.9998
Epoch 8/10
469/469 [==============================] - 78s 166ms/step - loss: 2.0448e-04 - accuracy: 1.0000
Epoch 9/10
469/469 [==============================] - 78s 167ms/step - loss: 1.1474e-04 - accuracy: 1.0000
Epoch 10/10
469/469 [==============================] - 78s 167ms/

In [ ]:
model7.evaluate(X_test, y_test)[1]

313/313 [==============================] - 7s 20ms/step - loss: 0.0184 - accuracy: 0.9951


0.9951000213623047

# 정리

epoch : 10 \\
batch_size = 128 \\
SGD(learning_rate=0.01, momentum=0.9) 사용


검증세트 사용, restore_best_weights = True : 0.9940999746322632 \\
검증세트 미사용 : 0.9951000213623047

